# Trabajo practico 1 procesamiento del habla
## Esteban Matias Cancino

Instalo las librerias necesarias:

In [1]:
!pip install pypdf requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 4.6 MB/s eta 0:00:00


Paso 1: Importo las bibliotecas necesarias
En este paso, importo las bibliotecas requeridas. Usare requests para descargar el PDF desde la URL proporcionada, pypdf (específicamente PdfReader) para extraer el texto del PDF, y re para manipular el texto con expresiones regulares

In [2]:
import requests
from pypdf import PdfReader
import re

Paso 2: Descargar el PDF desde la URL
uso requests para hacer una solicitud GET a la URL del PDF y guardo el contenido en un archivo local temporal llamado martin_fierro.pdf

In [3]:
# URL pública donde se encuentra el PDF de "El gaucho Martín Fierro".
url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"

# realizo una petición HTTP GET a la URL para descargar el recurso.
# `respuesta` contendrá el estado, headers y el contenido (entre otras cosas).
respuesta = requests.get(url)

# abro el archivo local "martin_fierro.pdf" en modo binario de escritura.
# uso `with` para que el archivo se cierre automáticamente al terminar el bloque.
with open("martin_fierro.pdf", "wb") as file:
    # se escribe en el archivo el contenido binario recibido en la respuesta HTTP.
    # `respuesta.content` contiene los bytes del PDF.
    file.write(respuesta.content)

# se imprime en consola un mensaje que indica que la descarga finalizó.
print("PDF descargado.")

PDF descargado.


Paso 3: Extraigo el texto completo del PDF usando PyPDF
Abro el PDF con PdfReader y extraigo el texto página por página usando extraction_mode="layout" para preservar el layout original, incluyendo separaciones entre estrofas. concateno todo el texto en una sola cadena (texto_completo), agrego prints de depuración para verificar la extracción

In [4]:
# creo un objeto PdfReader que abre y parsea el archivo PDF local "martin_fierro.pdf".
lector = PdfReader("martin_fierro.pdf")

# se inicializa una cadena vacia donde iremos acumulando todo el texto extraido de cada pagina.
texto_completo = ""

# se recorre cada pagina contenida en el PDF (lector.pages es un iterable de objetos pagina).
for pagina in lector.pages:
    # se extrae el texto de la pagina actual, el argumento extraction_mode="layout"
    # intenta preservar el diseño original (saltos de línea y disposición de párrafos)
    extraido = pagina.extract_text(extraction_mode="layout")
    # compruebo si se extrajo algún texto (es decir, que no sea None o una cadena vacia).
    if extraido:
        # si hay texto, lo añade al acumulador `texto_completo` y agrega un salto de línea
        # extra para separar claramente el contenido proveniente de paginas distintas.
        texto_completo += extraido + "\n"

# se indica por consola que el proceso de extraccion termino.
print("Texto extraído del PDF.")

# se muestra la longitud (en caracteres) del texto acumulado, lo cual es util para verificar rapidamente
# si se extrajo contenido (longitud > 0) o quedó vacío.
print("Longitud del texto extraído:", len(texto_completo))

Texto extraído del PDF.
Longitud del texto extraído: 72015


Paso 4: Limpio el texto para eliminar ruido, enfocandome en eliminar elementos no deseados como "Página X", números romanos solos (e.g., "II"), y títulos sueltos (e.g., "MARTIN FIERRO", "CRUZ") que aparecen como líneas independientes.

Primero, removere específicamente las secciones de páginas con sus \n adyacentes para evitar splits innecesarios en estrofas que cruzan páginas: usare re.sub para reemplazar "\nPágina \d+\n" por "\n", conectando versos adyacentes.
filtrare líneas: Eliminamos aquellas que marquen patrones como números romanos puros ("^[IVXLCDM]+$"), o líneas cortas en mayúsculas (títulos como "MARTIN FIERRO").
También eliminare líneas que son solo números.
normalizare múltiples saltos de línea a exactamente \n\n para separar estrofas.
buscare el inicio del poema ("Aquí me pongo a cantar") y corto desde allí.
quitando espacios extra al inicio/fin de cada línea, preservando líneas vacías para mantener las separaciones.
agregare algunos prints de depuración para ver el texto antes y después de cada limpieza.

In [5]:
# print inicial para ver el comienzo del texto antes de la limpieza
print("Texto antes de limpieza (primeros 500 chars):", texto_completo[:500])

Texto antes de limpieza (primeros 500 chars):                   El gaucho
               Martín Fierro






              José Hernández








Recursos de dominio público
                                            José Hernández
                                                    El Gaucho Martín Fierro









































José Hernández (1834 - 1886)
Imágenes de dominio público. Fuente:
http://upload.wikimedia.org/wikipedia/commons/8/8c/Jos%C3%A9_Hern%C3%A1ndez_Argentino.jpg
http://upload.wikimedia.org/wikipedia/co


In [6]:
# eliminar contenido antes del poema (buscando el inicio conocido)
print("Índice de inicio encontrado:", texto_completo.find("Aquí me pongo a cantar"))

# busco la posicion (indice) de la primera aparición del texto exacto
# "Aquí me pongo a cantar" dentro de `texto_completo`. `str.find()` devuelve:
# - el índice (0-based) donde empieza la coincidencia si la encuentra,
# - o -1 si no existe dicha subcadena.
# La siguiente línea recorta `texto_completo` empezando desde esa posición.
texto_completo = texto_completo[texto_completo.find("Aquí me pongo a cantar"):]

print("Después de stripping líneas (primeros 500 chars):", texto_completo[:500])

Índice de inicio encontrado: 1090
Después de stripping líneas (primeros 500 chars): Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.

Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.

Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.

Yo he visto muchos cantores,



In [7]:
# remover las líneas de "Página X" y sus \n adyacentes para conectar versos que cruzan páginas
texto_completo = re.sub(r'\n*\s*Página\s+\d+\s*\n*', '\n', texto_completo)

print("Después de remover páginas y conectar versos (primeros 500 chars):", texto_completo[:500])

print("Longitud del texto:", len(texto_completo))

Después de remover páginas y conectar versos (primeros 500 chars): Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.

Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.

Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.

Yo he visto muchos cantores,

Longitud del texto: 60450


In [8]:
# dividir en líneas para filtrar
lineas = texto_completo.split("\n")

# inicializa la lista donde almacenaremos las lineas que pasen los filtros.
lineas_filtradas = []

# se recorre cada linea obtenida tras el split por saltos de linea.
for linea in lineas:
    # se eliminan espacios en ambos extremos de la linea para normalizarla antes de evaluar.
    despojo = linea.strip()
    # compruebo si la linea no coincide con ninguno de los patrones a eliminar:
    #  - numeros romanos solos (ej. "II", "III")
    #  - titulos sueltos en mayusculas y cortos (<= 3 palabras)
    #  - lineas que sean solo números
    if (not re.match(r'^\s*[IVXLCDM]+\s*$', linea) and
        not (despojo.isupper() and len(despojo.split()) <= 3) and
        not re.match(r'^\s*\d+\s*$', linea)):
        # si la linea pasó todos los filtros, la añadimos a la lista final usando la version 'despojo'.
        lineas_filtradas.append(despojo)

In [9]:
# uno todas las lineas que quedaron en `lineas_filtradas` usando un salto de linea como separador,
# reconstruyendo asi el texto limpio en la variable `texto_completo`.
texto_completo = "\n".join(lineas_filtradas)

print("Después de filtrar líneas no deseadas (primeros 500 chars):", texto_completo[:500])

print("Longitud del texto:", len(texto_completo))

Después de filtrar líneas no deseadas (primeros 500 chars): Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.

Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.

Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.

Yo he visto muchos cantores,

Longitud del texto: 60268


In [10]:
# Reemplazar múltiples newlines por exactamente \n\n para separar estrofas
texto_completo = re.sub(r'\n{3,}', '\n\n', texto_completo)
print("Después de normalizar newlines (primeros 500 chars):", texto_completo[:500])
print("Texto limpiado. Longitud final:", len(texto_completo))

Después de normalizar newlines (primeros 500 chars): Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.

Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.

Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.

Yo he visto muchos cantores,

Texto limpiado. Longitud final: 60254


Paso 5: Divido el texto en estrofas usando re.split en \n\n+ (considerando variaciones en los saltos de línea). Para cada estrofa:

cuento la cantidad de versos (líneas no vacías),
construyo el texto uniendo los versos limpios con \n,
ignoro estrofas vacías o que parezcan residuales (e.g., cant_versos == 0 o == 1 si es un título residual, aunque ya los filtre en paso 4),
agrego prints de depuración para ver el número de estrofas y el primer elemento de la lista.

In [11]:
# separo el texto limpio en estrofas, uso r'\n\n+' para considerar dos o más saltos de linea
# como separador (maneja variaciones donde hay lineas en blanco dobles o triples)
estrofas = re.split(r'\n\n+', texto_completo.strip())

# se muestra cuantas estrofas detecto la division anterio
print("Numero de estrofas detectadas:", len(estrofas))

# se inicializa la lista que contendra diccionarios con la cantidad de versos y el texto de cada estrofa
lista_versos = []

# se recorre cada estrofa obtenida por el split anterior
for estrofa in estrofas:
    # se ignora estrofas que sean cadenas vacias o contengan solo espacios
    if estrofa.strip():
        # divido la estrofa en versos por salto de linea y normalizo cada verso con .strip()
        # ademas filtro versos vacios que puedan aparecer por lineas en blanco
        versos = [verso.strip() for verso in estrofa.split("\n") if verso.strip()]
        # evito incluir "residuales" que tengan un unico verso ya que pueden ser posibles cabeceras o ruido
        if versos and len(versos) > 1:
            # calculo la cantidad de versos que tiene la estrofa actual
            cant_versos = len(versos)
            # recontruyo el texto de la estrofa uniendo los versos con saltos de línea
            texto = "\n".join(versos)
            # añado a la lista_versos un diccionario con las claves solicitadas:
            # "cant_versos" -> número de versos, "texto" -> texto completo de la estrofa
            lista_versos.append({"cant_versos": cant_versos, "texto": texto})

print("Longitud de la lista de versos generada:", len(lista_versos))

print("Primer elemento de la lista de versos:", lista_versos[0])

Numero de estrofas detectadas: 370
Longitud de la lista de versos generada: 370
Primer elemento de la lista de versos: {'cant_versos': 6, 'texto': 'Aquí me pongo a cantar\nal compás de la vigüela,\nque el hombre que lo desvela\nuna pena estrordinaria,\ncomo la ave solitaria\ncon el cantar se consuela.'}


In [12]:
print("Verifico el ultimo elemento de la lista de versos",lista_versos[-1])

Verifico el ultimo elemento de la lista de versos {'cant_versos': 6, 'texto': 'Pero ponga su esperanza\nen el Dios que lo formó;\ny aquí me despido yo,\nque referí ansí a mi modo\nMALES QUE CONOCEN TODOS\nPERO QUE NAIDES CONTO.'}


In [13]:
# uso un for para recorrer la lista
for i in range(len(lista_versos)):
  print(i)
  print(lista_versos[i])

0
{'cant_versos': 6, 'texto': 'Aquí me pongo a cantar\nal compás de la vigüela,\nque el hombre que lo desvela\nuna pena estrordinaria,\ncomo la ave solitaria\ncon el cantar se consuela.'}
1
{'cant_versos': 6, 'texto': 'Pido a los santos del cielo\nque ayuden mi pensamiento:\nles pido en este momento\nque voy a cantar mi historia\nme refresquen la memoria\ny aclaren mi entendimiento.'}
2
{'cant_versos': 6, 'texto': 'Vengan santos milagrosos,\nvengan todos en mi ayuda,\nque la lengua se me añuda\ny se me turba la vista;\npido a mi Dios que me asista\nen una ocasión tan ruda.'}
3
{'cant_versos': 6, 'texto': 'Yo he visto muchos cantores,\ncon famas bien otenidas\ny que después de alquiridas\nno las quieren sustentar:\nparece que sin largar\nse cansaron en partidas.'}
4
{'cant_versos': 6, 'texto': 'Mas ande otro criollo pasa\nMartín Fierro ha de pasar;\nnada lo hace recular,\nni las fantasmas lo espantan,\ny dende que todos cantan\nyo también quiero cantar.'}
5
{'cant_versos': 6, 'texto': '

# Conclusiones
El trabajo muestra un proceso completo de extracción y preprocesamiento de texto a partir de un archivo PDF, en este caso, "El gaucho Martín Fierro" de José Hernández. La implementación de bibliotecas como requests y pypdf me permitió automatizar la descarga y la lectura del documento, mientras que el uso de expresiones regulares con la biblioteca re fue fundamental para limpiar el texto, eliminando encabezados, numeración de páginas y otros ruidos, que podrian afectar la construcción de la lista de versos.

Referencias: la fuente (el pdf) y las herramientas que use

José Hernández. (1872). El gaucho Martín Fierro. Obtenido de https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf.


libreria requests. Para la descarga de archivos a través de URL.


libreria pypdf. Para la extracción de texto de archivos PDF.


modulo re. Para la manipulación y limpieza de texto
